### Importar Arquivos e Bibliotecas

In [ ]:
import pandas as pd
import pathlib
import yagmail

emails = pd.read_excel(r'Bases de Dados/Emails.xlsx')
lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin1', sep=';')
vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

#Usuário e Senha para enviar o e-mail
usuario = yagmail.SMTP(user='xxx@xxx.com', password='xxxxxxxxx')

### Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [ ]:
# Incluir nome da loja na variável vendas

vendas = vendas.merge(lojas, on='ID Loja')
display(vendas)

In [ ]:
# Criar dicionário de lojas

dic_lojas = {}
for loja in lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja']==loja, :]
    
display(dic_lojas['Norte Shopping'])
display(dic_lojas['Salvador Shopping'])

In [ ]:
# Definir a váriavel dia_indicador com a última data da tabela

dia_indicador = vendas['Data'].max()
print(dia_indicador)

### Salvar a planilha na pasta de backup

In [ ]:
# Verificar se a pasta já existe

caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_pasta_backup = caminho_backup.iterdir()

lista_nomes_backup = [arquivo.name for arquivo in arquivos_pasta_backup]

for loja in dic_lojas:
    if loja not in lista_nomes_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    
    # Salvar dentro da pasta
    
    nome_arquivo = '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dic_lojas[loja].to_excel(local_arquivo)

### Calcular o indicador

In [ ]:
# Definir as metas

meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtd_produtos_dia = 4
meta_qtd_produtos_ano = 120
meta_tkm_dia = 500
meta_tkm_ano = 500

In [ ]:
for loja in dic_lojas:
    vendas_loja = dic_lojas[loja]
    vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

    # Faturamento

    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_loja_dia['Valor Final'].sum()

    # Diversidade de produtos

    qtd_produtos_ano = len(vendas_loja['Produto'].unique())
    qtd_produtos_dia = len(vendas_loja_dia['Produto'].unique())

    # Ticket Médio Ano

    valor_venda = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda['Valor Final'].mean()

    # Ticket Médio Ano

    valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
    
    
    #print(faturamento_ano)
    #print(faturamento_dia)
    #print(qtd_produtos_ano)
    #print(qtd_produtos_dia)
    #print(ticket_medio_ano)
    #print(ticket_medio_dia)
    
    ## ENVIAR E-MAIL ##
    
    nome = emails.loc[emails['Loja']==loja, 'Gerente'].values[0]

    if faturamento_dia >= meta_faturamento_dia:
        cor_fat_dia = 'green'
    else:
        cor_fat_dia = 'red'

    if faturamento_ano >= meta_faturamento_ano:
        cor_fat_ano = 'green'
    else:
        cor_fat_ano = 'red'

    if qtd_produtos_dia >= meta_qtd_produtos_dia:
        cor_qtde_dia = 'green'
    else:
        cor_qtde_dia = 'red'

    if qtd_produtos_ano >= meta_qtd_produtos_ano:
        cor_qtde_ano = 'green'
    else:
        cor_qtde_ano = 'red'

    if ticket_medio_dia >= meta_tkm_dia:
        cor_ticket_dia = 'green'
    else:
        cor_ticket_dia = 'red'

    if ticket_medio_ano >= meta_tkm_ano:
        cor_ticket_ano = 'green'
    else:
        cor_ticket_ano = 'red'

    corpo_email = f'''
        <p">Bom dia {nome},</p>
        <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da <strong>loja {loja}</strong> foi: </p>
        <br>
        <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
          </tr>
            <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_dia:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
          </tr>
          <tr>
            <td>Diversidade De Produtos</td>
            <td style="text-align: center">{qtd_produtos_dia}</td>
            <td style="text-align: center">{meta_qtd_produtos_dia}</td>
            <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
            <td style="text-align: center">R${meta_tkm_dia:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
          </tr>
        </table>
        <br>
        <br>
            <table>
          <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
          </tr>
            <tr>
            <td>Faturamento</td>
            <td style="text-align: center">R${faturamento_ano:.2f}</td>
            <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
          </tr>
          <tr>
            <td>Diversidade De Produtos</td>
            <td style="text-align: center">{qtd_produtos_ano}</td>
            <td style="text-align: center">{meta_qtd_produtos_ano}</td>
            <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
          </tr>
          <tr>
            <td>Ticket Médio</td>
            <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
            <td style="text-align: center">R${meta_tkm_ano:.2f}</td>
            <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
          </tr>
        </table>
        <p>Segue em anexo a planilha com todos os dados para mais detalhes.</p>
        <p>Att., Raphael.</p>
    '''

    ## Devido bug no yagmail é necessário trocar toda quebra de linha "\n" por "" para não termos  ##
    ##  linhas em branco no corpo do email.                                                        ##
    corpo_email = corpo_email.replace("\n", "")

    usuario.send(to=emails.loc[emails['Loja']==loja, 'E-mail'].values[0],
                 subject='OnePage Dia {}/{} - Loja {}'.format(dia_indicador.day, dia_indicador.month, loja),
                 contents=corpo_email, 
                 attachments=pathlib.Path.cwd() / caminho_backup / loja / '{}_{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.day, loja)
                )

In [ ]:
# Gerar ranking ano
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_ano = faturamento_lojas.sort_values(by='Valor Final', ascending=False)

#Gerar ranking dia
vendas_dia = vendas.loc[vendas['Data']==dia_indicador, :]
faturamento_lojas_dia = vendas_dia.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by='Valor Final', ascending=False)

#Criar arquivos
nome_arquivo = '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_ano.to_excel(caminho_backup / nome_arquivo)

nome_arquivo = '{}_{}_Ranking Diario.xlsx'.format(dia_indicador.month, dia_indicador.day)
faturamento_lojas_dia.to_excel(caminho_backup / nome_arquivo)

#display(faturamento_lojas_ano)
#display(faturamento_lojas_dia)

### Enviar e-mail para diretoria

In [ ]:
corpo_email = f'''
Prezados, bom dia.

Melhor loja do dia em faturamento: Loja <strong>{faturamento_lojas_dia.index[0]}</strong> com faturamento: <strong>R$ {faturamento_lojas_dia.iloc[0, 0]:.2f}</strong>
Pior loja do dia em faturamento: Loja <strong>{faturamento_lojas_dia.index[-1]}</strong> com faturamento: <strong>R$ {faturamento_lojas_dia.iloc[-1, -1]:.2f}</strong>

Melhor loja do ano em faturamento: Loja <strong>{faturamento_lojas_ano.index[0]}</strong> com faturamento: <strong>R$ {faturamento_lojas_ano.iloc[0, 0]:.2f}</strong>
Pior loja do ano em faturamento: Loja <strong>{faturamento_lojas_ano.index[-1]}</strong> com faturamento: <strong>R$ {faturamento_lojas_ano.iloc[-1, -1]:.2f}</strong>

Segue em anexo os ranking do ano e do dia de todas as lojas.

Qualquer duvida estou a disposição.

Att., Raphael.
'''

usuario.send(to=emails.loc[emails['Loja']=='Diretoria', 'E-mail'].values[0],
             subject='Ranking Dia {}/{}'.format(dia_indicador.day, dia_indicador.month),
             contents=corpo_email, 
             attachments=[pathlib.Path.cwd() / caminho_backup / '{}_{}_Ranking Anual.xlsx'.format(dia_indicador.month, dia_indicador.day), pathlib.Path.cwd() / caminho_backup / '{}_{}_Ranking Diario.xlsx'.format(dia_indicador.month, dia_indicador.day)]
            )